In [1]:
from gevent import monkey
monkey.patch_all(thread=False)
from gevent.queue import Queue
import gevent

import sys
sys.path.append('../../../00.module')
import UA
import Proxy

# = = = = = = = = = = = = = = =

import pandas as pd

output_correct = pd.DataFrame()

output_error = pd.DataFrame()

input_ = pd.read_excel('./2.category.xlsx',
                       header=0,
                       dtype=str).fillna('')

length = len(input_)

print('总数量：' + str(length))
print()

# = = = = = = = = = = = = = = =

import requests
import json

from bs4 import BeautifulSoup

from lxml import etree

# = = = = = = = = = = = = = = =

import time
from datetime import datetime

# = = = = = = = = = = = = = = =

work = Queue()
for i in range(length):
    work.put_nowait(i)

def crawler():
    global output_correct, output_error
    
    while not work.empty():
        i = work.get_nowait()
        
        # = = = = = = = = = = = = = = =
        
        crawler_status = 'error'
        for _ in range(101):
            try:
                payload = {'func': 'partsearchgetparttypessdd',
                           'mfr': input_.loc[i, 'Manufacturer'],
                           'pg': input_.loc[i, 'Category'],
                           'api_json_request': '1'}

                resp = requests.post('https://www.rockauto.com/catalog/catalogapi.php',
                                     data=payload,
                                     headers=UA.get_User_Agent_Requests(),
                                     proxies=Proxy.get_Proxy_Requests()).text

                resp = json.loads(resp)
                
                # = = = = = = = = = = = = = = =
                
                html = resp['html_replace_sections']['ddrepl[parttype_][container]']

                soup = BeautifulSoup(html, 'lxml')
                
                # = = = = = = = = = = = = = = =
                
                html = etree.HTML(str(soup))

                list_type = [type_.strip() for type_ in html.xpath('//select[@id="parttype_"]/option/text()')[1:]]

                # = = = = = = = = = = = = = = =
                
                list_type_code = [type_code.strip() for type_code in html.xpath('//select[@id="parttype_"]/option/@value')[1:]]
                
                # = = = = = = = = = = = = = = =
                
                crawler_status = 'ok'
                
                break
                
            except:
                time.sleep(0.3)
                continue

        if crawler_status == 'ok':
            df_temp = pd.DataFrame({'Tab_Label': input_.loc[i, 'Tab_Label'],
                                    'No.': int(input_.loc[i, 'No.']),
                                    'Manufacturer': input_.loc[i, 'Manufacturer'],
                                    'Category': input_.loc[i, 'Category'],
                                    'Type': list_type,
                                    'Type_Code': list_type_code,
                                    'Url': ['https://www.rockauto.com/en/parts/'+input_.loc[i, 'Manufacturer'].replaceace('+', '%2B').replace(',', '%2C')+','+type_.replace(',', '%2C')+','+type_code for type_, type_code in zip(list_type, list_type_code)]})

            output_correct = pd.concat([output_correct, df_temp], ignore_index=True).fillna('')
        else:
            df_temp = pd.DataFrame([{'Tab_Label': input_.loc[i, 'Tab_Label'],
                                     'No.': int(input_.loc[i, 'No.']),
                                     'Manufacturer': input_.loc[i, 'Manufacturer'],
                                     'Category': input_.loc[i, 'Category']}])
    
            output_error = pd.concat([output_error, df_temp], ignore_index=True).fillna('')

        print(input_.loc[i, 'Manufacturer'] + ' - ' + input_.loc[i, 'Category'] +  '  <->  [' + crawler_status + '] - 剩余数量：' + str(work.qsize()) + ' - 当前时间：' + datetime.now().strftime('%H:%M:%S'))

# = = = = = = = = = = = = = = =

list_task = []
for _ in range(20):
    task = gevent.spawn(crawler)
    list_task.append(task)
gevent.joinall(list_task)

print()
print('爬虫输出中')
output_correct = output_correct.drop_duplicates(ignore_index=True)
output_correct = output_correct.sort_values(by=['No.', 'Category', 'Type'], ascending=[True, True, True])
output_correct.to_excel('./3.type-' + datetime.now().strftime('%Y%m%d_%H%M%S') + '.xlsx', index=False)
if len(output_error) != 0:
    output_error = output_error.drop_duplicates(ignore_index=True)
    output_error = output_error.sort_values(by=['No.', 'Category'], ascending=[True, True])
    output_error.to_excel('./type_error.xlsx', index=False)
    print()
    print('爬虫存在error')
print()
print('爬虫结束')

总数量：2472

3D - Interior  <->  [ok] - 剩余数量：2467 - 当前时间：18:03:36
3D - Body & Lamp Assembly  <->  [ok] - 剩余数量：2466 - 当前时间：18:03:37
3M - Body & Lamp Assembly  <->  [ok] - 剩余数量：2465 - 当前时间：18:03:37
3M - Fuel & Air  <->  [ok] - 剩余数量：2464 - 当前时间：18:03:37
3M - Garage Equipment  <->  [ok] - 剩余数量：2463 - 当前时间：18:03:38
AAE - Steering  <->  [ok] - 剩余数量：2462 - 当前时间：18:03:39
ACC - Interior  <->  [ok] - 剩余数量：2461 - 当前时间：18:03:39
3M - Interior  <->  [ok] - 剩余数量：2460 - 当前时间：18:03:40
ACDELCO - Belt Drive  <->  [ok] - 剩余数量：2459 - 当前时间：18:03:40
ACDELCO - Body & Lamp Assembly  <->  [ok] - 剩余数量：2458 - 当前时间：18:03:40
ACDELCO - Cooling System  <->  [ok] - 剩余数量：2457 - 当前时间：18:03:41
ACCEL - Ignition  <->  [ok] - 剩余数量：2456 - 当前时间：18:03:41
ACDELCO - Electrical  <->  [ok] - 剩余数量：2455 - 当前时间：18:03:42
ACDELCO - Brake & Wheel Hub  <->  [ok] - 剩余数量：2454 - 当前时间：18:03:43
ACDELCO - Electrical-Switch & Relay  <->  [ok] - 剩余数量：2453 - 当前时间：18:03:44
ACDELCO - Electrical-Connector  <->  [ok] - 剩余数量：2452 - 当前时间：18:03:44
ACDELCO 

AMERICAN COMPONENTS - Transmission-Manual  <->  [ok] - 剩余数量：2338 - 当前时间：18:04:46
API - Belt Drive  <->  [ok] - 剩余数量：2337 - 当前时间：18:04:46
API - Body & Lamp Assembly  <->  [ok] - 剩余数量：2336 - 当前时间：18:04:47
API - Brake & Wheel Hub  <->  [ok] - 剩余数量：2335 - 当前时间：18:04:47
AMS AUTOMOTIVE - Body & Lamp Assembly  <->  [ok] - 剩余数量：2334 - 当前时间：18:04:47
API - Drivetrain  <->  [ok] - 剩余数量：2333 - 当前时间：18:04:48
API - Cooling System  <->  [ok] - 剩余数量：2332 - 当前时间：18:04:48
API - Electrical-Connector  <->  [ok] - 剩余数量：2331 - 当前时间：18:04:49
API - Electrical-Bulb & Socket  <->  [ok] - 剩余数量：2330 - 当前时间：18:04:50
API - Electrical-Switch & Relay  <->  [ok] - 剩余数量：2329 - 当前时间：18:04:50
APEX - Ignition  <->  [ok] - 剩余数量：2328 - 当前时间：18:04:50
API - Exhaust & Emission  <->  [ok] - 剩余数量：2327 - 当前时间：18:04:51
API - Fuel & Air  <->  [ok] - 剩余数量：2326 - 当前时间：18:04:51
API - Engine  <->  [ok] - 剩余数量：2325 - 当前时间：18:04:52
API - Heat & Air Conditioning  <->  [ok] - 剩余数量：2324 - 当前时间：18:04:52
API - Ignition  <->  [ok] - 剩余数量：2323 

AUTOSPECIALTY/KELSEY-HAYES - Steering  <->  [ok] - 剩余数量：2209 - 当前时间：18:06:27
AUTOSPECIALTY/KELSEY-HAYES - Brake & Wheel Hub  <->  [ok] - 剩余数量：2208 - 当前时间：18:06:28
AUTOSPECIALTY/KELSEY-HAYES - Transmission-Manual  <->  [ok] - 剩余数量：2207 - 当前时间：18:06:28
AUTOMETER - Interior  <->  [ok] - 剩余数量：2206 - 当前时间：18:06:28
AUTOTECH ENGINEERING - Drivetrain  <->  [ok] - 剩余数量：2205 - 当前时间：18:06:29
AUTOTECNICA - Belt Drive  <->  [ok] - 剩余数量：2204 - 当前时间：18:06:31
AUTOTECNICA - Body & Lamp Assembly  <->  [ok] - 剩余数量：2203 - 当前时间：18:06:32
AUTOTECNICA - Drivetrain  <->  [ok] - 剩余数量：2202 - 当前时间：18:06:32
AUTOTECNICA - Cooling System  <->  [ok] - 剩余数量：2201 - 当前时间：18:06:32
AUTOTECNICA - Electrical  <->  [ok] - 剩余数量：2200 - 当前时间：18:06:34
AUTOTECNICA - Electrical-Connector  <->  [ok] - 剩余数量：2199 - 当前时间：18:06:35
AUTOTECNICA - Electrical-Bulb & Socket  <->  [ok] - 剩余数量：2198 - 当前时间：18:06:35
AUTOTECNICA - Electrical-Switch & Relay  <->  [ok] - 剩余数量：2197 - 当前时间：18:06:36
AUTOTECNICA - Exhaust & Emission  <->  [ok] - 剩余数量：

BRUTE POWER - Electrical-Connector  <->  [ok] - 剩余数量：2083 - 当前时间：18:08:05
BRUTE POWER - Cooling System  <->  [ok] - 剩余数量：2082 - 当前时间：18:08:06
BOSCH - Steering  <->  [ok] - 剩余数量：2081 - 当前时间：18:08:07
BRUTE POWER - Transmission-Manual  <->  [ok] - 剩余数量：2080 - 当前时间：18:08:08
BRUTE POWER - Exhaust & Emission  <->  [ok] - 剩余数量：2079 - 当前时间：18:08:08
BRUTE POWER - Fuel & Air  <->  [ok] - 剩余数量：2078 - 当前时间：18:08:09
BRUTE POWER - Engine  <->  [ok] - 剩余数量：2077 - 当前时间：18:08:09
BSE - Brake & Wheel Hub  <->  [ok] - 剩余数量：2076 - 当前时间：18:08:10
BSE - Electrical-Switch & Relay  <->  [ok] - 剩余数量：2075 - 当前时间：18:08:10
BSE - Drivetrain  <->  [ok] - 剩余数量：2074 - 当前时间：18:08:11
BSE - Exhaust & Emission  <->  [ok] - 剩余数量：2073 - 当前时间：18:08:12
BSE - Fuel & Air  <->  [ok] - 剩余数量：2072 - 当前时间：18:08:12
BSE - Ignition  <->  [ok] - 剩余数量：2071 - 当前时间：18:08:13
BSE - Electrical  <->  [ok] - 剩余数量：2070 - 当前时间：18:08:13
BSE - Heat & Air Conditioning  <->  [ok] - 剩余数量：2069 - 当前时间：18:08:13
BSE - Interior  <->  [ok] - 剩余数量：2068 - 当前时间

CONTINENTAL - Wiper & Washer  <->  [ok] - 剩余数量：1959 - 当前时间：18:09:45
CONTINENTAL - Heat & Air Conditioning  <->  [ok] - 剩余数量：1958 - 当前时间：18:09:47
CORTECO - Drivetrain  <->  [ok] - 剩余数量：1957 - 当前时间：18:09:48
CORTECO - Cooling System  <->  [ok] - 剩余数量：1956 - 当前时间：18:09:50
CORTECO - Electrical  <->  [ok] - 剩余数量：1955 - 当前时间：18:09:51
CORTECO - Exhaust & Emission  <->  [ok] - 剩余数量：1954 - 当前时间：18:09:52
CORTECO - Engine  <->  [ok] - 剩余数量：1953 - 当前时间：18:09:53
CORTECO - Fuel & Air  <->  [ok] - 剩余数量：1952 - 当前时间：18:09:53
CORTECO - Ignition  <->  [ok] - 剩余数量：1951 - 当前时间：18:09:54
CORTECO - Transmission-Automatic  <->  [ok] - 剩余数量：1950 - 当前时间：18:09:56
CORTECO - Steering  <->  [ok] - 剩余数量：1949 - 当前时间：18:09:57
CONTINENTAL - Engine  <->  [ok] - 剩余数量：1948 - 当前时间：18:09:57
COVERLAY - Interior  <->  [ok] - 剩余数量：1947 - 当前时间：18:09:59
CROWN REMANUFACTURING - Heat & Air Conditioning  <->  [ok] - 剩余数量：1946 - 当前时间：18:09:59
CONTINENTAL - Transmission-Automatic  <->  [ok] - 剩余数量：1945 - 当前时间：18:10:01
CORTECO - Transmi

...

SHEE-MAR - Electrical-Bulb & Socket  <->  [ok] - 剩余数量：561 - 当前时间：18:24:28
SHEE-MAR - Electrical-Connector  <->  [ok] - 剩余数量：560 - 当前时间：18:24:28
SHEE-MAR - Electrical-Switch & Relay  <->  [ok] - 剩余数量：559 - 当前时间：18:24:28
SHEE-MAR - Exhaust & Emission  <->  [ok] - 剩余数量：558 - 当前时间：18:24:29
SCHAEFFLER - Electrical  <->  [ok] - 剩余数量：557 - 当前时间：18:24:31
SHEE-MAR - Engine  <->  [ok] - 剩余数量：556 - 当前时间：18:24:31
SHEE-MAR - Heat & Air Conditioning  <->  [ok] - 剩余数量：555 - 当前时间：18:24:32
SHEE-MAR - Ignition  <->  [ok] - 剩余数量：554 - 当前时间：18:24:32
SHEE-MAR - Steering  <->  [ok] - 剩余数量：553 - 当前时间：18:24:33
SHEE-MAR - Interior  <->  [ok] - 剩余数量：552 - 当前时间：18:24:34
SHEE-MAR - Suspension  <->  [ok] - 剩余数量：551 - 当前时间：18:24:35
SHEE-MAR - Transmission-Automatic  <->  [ok] - 剩余数量：550 - 当前时间：18:24:35
SILLA COOLING SYSTEM - Heat & Air Conditioning  <->  [ok] - 剩余数量：549 - 当前时间：18:24:37
SHEE-MAR - Transmission-Manual  <->  [ok] - 剩余数量：548 - 当前时间：18:24:38
SILV-O-LITE - Engine  <->  [ok] - 剩余数量：547 - 当前时间：18:24:38
SHE

STRONGARM - Interior  <->  [ok] - 剩余数量：436 - 当前时间：18:26:00
SUBARU - Body & Lamp Assembly  <->  [ok] - 剩余数量：435 - 当前时间：18:26:00
SUBARU - Cooling System  <->  [ok] - 剩余数量：434 - 当前时间：18:26:01
STANDARD MOTOR PRODUCTS - Electrical  <->  [ok] - 剩余数量：433 - 当前时间：18:26:02
SUBARU - Electrical  <->  [ok] - 剩余数量：432 - 当前时间：18:26:02
SUBARU - Electrical-Bulb & Socket  <->  [ok] - 剩余数量：431 - 当前时间：18:26:02
SUBARU - Drivetrain  <->  [ok] - 剩余数量：430 - 当前时间：18:26:03
SUBARU - Electrical-Connector  <->  [ok] - 剩余数量：429 - 当前时间：18:26:03
SUBARU - Engine  <->  [ok] - 剩余数量：428 - 当前时间：18:26:03
SUBARU - Electrical-Switch & Relay  <->  [ok] - 剩余数量：427 - 当前时间：18:26:04
SUBARU - Exhaust & Emission  <->  [ok] - 剩余数量：426 - 当前时间：18:26:04
SUBARU - Fuel & Air  <->  [ok] - 剩余数量：425 - 当前时间：18:26:04
SUBARU - Hardware  <->  [ok] - 剩余数量：424 - 当前时间：18:26:05
SUBARU - Heat & Air Conditioning  <->  [ok] - 剩余数量：423 - 当前时间：18:26:05
SUBARU - Interior  <->  [ok] - 剩余数量：422 - 当前时间：18:26:06
SUBARU - Steering  <->  [ok] - 剩余数量：421 - 当前时间

ULTRA-POWER - Body & Lamp Assembly  <->  [ok] - 剩余数量：306 - 当前时间：18:27:00
ULTRA-POWER - Cooling System  <->  [ok] - 剩余数量：305 - 当前时间：18:27:01
ULTRA-POWER - Drivetrain  <->  [ok] - 剩余数量：304 - 当前时间：18:27:01
ULTRA-POWER - Brake & Wheel Hub  <->  [ok] - 剩余数量：303 - 当前时间：18:27:01
ULTRA-POWER - Electrical-Connector  <->  [ok] - 剩余数量：302 - 当前时间：18:27:02
ULTRA-POWER - Electrical-Bulb & Socket  <->  [ok] - 剩余数量：301 - 当前时间：18:27:02
ULTRA-POWER - Engine  <->  [ok] - 剩余数量：300 - 当前时间：18:27:03
ULTRA-POWER - Electrical-Switch & Relay  <->  [ok] - 剩余数量：299 - 当前时间：18:27:03
ULTRA-POWER - Fuel & Air  <->  [ok] - 剩余数量：298 - 当前时间：18:27:05
ULTRA-POWER - Heat & Air Conditioning  <->  [ok] - 剩余数量：297 - 当前时间：18:27:06
ULTRA-POWER - Exhaust & Emission  <->  [ok] - 剩余数量：296 - 当前时间：18:27:07
ULTRA-POWER - Hoses/Lines & Clamps  <->  [ok] - 剩余数量：295 - 当前时间：18:27:07
ULTRA-POWER - Ignition  <->  [ok] - 剩余数量：294 - 当前时间：18:27:08
UBP - Transmission-Manual  <->  [ok] - 剩余数量：293 - 当前时间：18:27:08
UBP - Brake & Wheel Hub  <->  [o

VARIOUS MFR - Interior  <->  [ok] - 剩余数量：183 - 当前时间：18:27:59
VARIOUS MFR - Steering  <->  [ok] - 剩余数量：182 - 当前时间：18:27:59
VARIOUS MFR - Fuel & Air  <->  [ok] - 剩余数量：181 - 当前时间：18:27:59
VARIOUS MFR - Suspension  <->  [ok] - 剩余数量：180 - 当前时间：18:28:00
VARIOUS MFR - Transmission-Automatic  <->  [ok] - 剩余数量：179 - 当前时间：18:28:00
VARIOUS MFR - Wheel  <->  [ok] - 剩余数量：178 - 当前时间：18:28:00
VARIOUS MFR - Wiper & Washer  <->  [ok] - 剩余数量：177 - 当前时间：18:28:01
VC - Belt Drive  <->  [ok] - 剩余数量：176 - 当前时间：18:28:01
VC - Electrical  <->  [ok] - 剩余数量：175 - 当前时间：18:28:02
VC - Brake & Wheel Hub  <->  [ok] - 剩余数量：174 - 当前时间：18:28:02
VC - Wiper & Washer  <->  [ok] - 剩余数量：173 - 当前时间：18:28:02
VDO / CONTINENTAL - Brake & Wheel Hub  <->  [ok] - 剩余数量：172 - 当前时间：18:28:03
VDO / CONTINENTAL - Cooling System  <->  [ok] - 剩余数量：171 - 当前时间：18:28:03
VDO / CONTINENTAL - Electrical  <->  [ok] - 剩余数量：170 - 当前时间：18:28:04
VDO / CONTINENTAL - Electrical-Switch & Relay  <->  [ok] - 剩余数量：169 - 当前时间：18:28:04
VDO / CONTINENTAL - Bod

WALKER PRODUCTS - Heat & Air Conditioning  <->  [ok] - 剩余数量：56 - 当前时间：18:29:17
WESTAR - Engine  <->  [ok] - 剩余数量：55 - 当前时间：18:29:18
WESTAR - Suspension  <->  [ok] - 剩余数量：54 - 当前时间：18:29:18
WESTAR - Transmission-Manual  <->  [ok] - 剩余数量：53 - 当前时间：18:29:19
WESTAR - Transmission-Automatic  <->  [ok] - 剩余数量：52 - 当前时间：18:29:19
WESTIN - Body & Lamp Assembly  <->  [ok] - 剩余数量：51 - 当前时间：18:29:19
WESTIN - Electrical  <->  [ok] - 剩余数量：50 - 当前时间：18:29:20
WESTIN - Electrical-Bulb & Socket  <->  [ok] - 剩余数量：49 - 当前时间：18:29:21
WESTIN - Interior  <->  [ok] - 剩余数量：48 - 当前时间：18:29:21
WESTIN - Wheel  <->  [ok] - 剩余数量：47 - 当前时间：18:29:22
WESTLING - Wiper & Washer  <->  [ok] - 剩余数量：46 - 当前时间：18:29:23
WHITE KNIGHT - Wheel  <->  [ok] - 剩余数量：45 - 当前时间：18:29:23
WIX - Engine  <->  [ok] - 剩余数量：44 - 当前时间：18:29:24
WIX - Cooling System  <->  [ok] - 剩余数量：43 - 当前时间：18:29:24
WIX - Fuel & Air  <->  [ok] - 剩余数量：42 - 当前时间：18:29:26
WIX - Exhaust & Emission  <->  [ok] - 剩余数量：41 - 当前时间：18:29:26
WALKER PRODUCTS - Fuel & Air 